## Shmuel Ruppo - 
## Analyzing Global Terrorist Attacks

<pre>In the GTD database, the groups who carried out some of the terror attacks have not been 
identified. The task is to build a model that predicts the responsible group of those 
unidentified attacks</pre>

In [1]:
import pandas as pd
import numpy as np
import time
import h2o
from h2o.estimators import H2ORandomForestEstimator
import random

<pre>
Importing a large file in CSV format is 10-20 times faster than importing an Excel file.
Using MS Excel, I have converted the Excel file provided to CSV. </pre>

In [2]:
# read from file
df = pd.read_csv('data/gtd/ALL_DATASET.csv', header=0, low_memory=False )
df.shape

(181691, 135)

   ## Data cleaning:

<pre>Throughout all the columns, the values -99, 9 stand for the 'Unknown' category.
Therefore, they all can be replaced with nan </pre>

In [3]:
df = df.replace(to_replace=[-99, -9], value=np.nan)


<pre>
Using the codebook provided by GTD (https://www.start.umd.edu/gtd/downloads/Codebook.pdf),
and double-checking through the excel file, I have learned that every textual categorical
variable in the dataset is already encoded as a numerical categorical variable.
For example, the list of countries : 'country_txt' is encoded as a numeric list of countries
'country'. Thus, I can work only with numeric variables. 
But, an important exception is the name of the terror group 'gname', which is not encoded.
So, I leave it as is. </pre>

In [4]:
df = df._get_numeric_data().join(df['gname'])

#Also, 'eventid' uniquely identifies each terror act. But it does not carry any meaning within it,
#and therefore can be dropped

df = df.drop('eventid', axis=1)



<pre>The name of the terror group is the response variable.
Terror groups that have been identified a small amount of times will likely will add more
noise than information. Thus, I remove terror groups that have carried out less (or equal) to 5
terror acts. </pre>

In [5]:
temp = ( pd.value_counts(df.loc[:,'gname']))
df = df.loc[df['gname'].isin(temp.index[temp > 5]),:]


# Display the dimensions of the data
df.shape

(177093, 77)

<pre>I distinguish between numeric and categorical variables, making a list of each


In [6]:
#List of the numeric variables:
numericList = [
 'nkill', 'nkillus', 'nkillter', 'nwound', 'nwoundus', 
    'nwoundte', 'nhostkid', 'iyear', 'imonth', 'iday','nhostkidus',  'ransomamt', 'ransomamtus',
'ransompaid', 'ransompaidus', 'nreleased', 'propvalue', 'latitude', 'longitude']


# Get the location of numeric
locNumeric = [df.columns.get_loc(c) for c in numericList]

#Whatever is not numeric, is categoric
categNum = [i for i in range(len(df.columns)) if i not in locNumeric]
categList = [i for i in df.columns if i not in numericList]

<pre>I separate, and put  aside the terror acts that have not been identified as belonging to a 
specific terror group. I will use them for the final prediction</pre>

In [7]:
mask_index = df.loc[(df['gname'] == 'Unknown'), 'gname'].index

prd_group = df.loc[mask_index,:]   # Will be used for the final prediction
df = df.drop(mask_index)

# The model : Random Forest

<pre>I will use the Random Forest method for classification of the data.
The Random Forest method is possible to use with a large amount of features.
It is highly economical in terms of preprocessing. It handles both categorical 
and numerical variables. (The numerical data does not even need scaling). 

Rather than guessing \ deciding using an expert opinion (which is unavailable to me)
what are the most important features, it is possible to analyze the features that contribute
the most to the Random Forest technique.

My strategy is:
First, use a Random Forest model on a small subset of the database.
See what are the features that contribute most to the random forest.
Then use those features on a larger subset of the database.

I will use the H2O library. 
A benefit of using H2O library is its efficient use of categorical variables.
It deals with categorical variables without needing one-hot encoding. </pre>

In [8]:
t=time.clock()

# Differentiate between training and response variables
training_columns=list(df.columns.difference(['gname']))
response_column = ['gname']

# Create sample the data
random.seed(123)
mysample = random.sample(range(df.shape[0]), 10000)

# init H2O
h2o.init()
h2o.remove_all()

# convert the data into H2OFrame
hf = h2o.H2OFrame(df.loc[mysample,:])

# Distinguish between categoric and numeric features
hf[:,numericList]=hf[:,numericList].asnumeric() 
hf[:,categList] = hf[:,categList].asfactor()

## Split the data into a train and a test set.  Random Forest will be tested on the latter.
train, test = hf.split_frame(ratios=[0.8], seed = 123)


# Do not select a large number of trees grown, and too large a depth initially
model = H2ORandomForestEstimator(  max_depth=5, ntrees = 5, seed= 123)

# Train model

model.train(x=training_columns, y=str(response_column[0]), training_frame=train)

# Test the model
performance = model.model_performance(test_data=test)
print 'seconds it took', time.clock() - t
print performance.hit_ratio_table()     

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "10.0.2" 2018-07-17; OpenJDK Runtime Environment (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4); OpenJDK 64-Bit Server VM (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4, mixed mode)
  Starting server from /usr/local/lib/python2.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpVsfDwC
  JVM stdout: /tmp/tmpVsfDwC/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpVsfDwC/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.3
H2O cluster version age:,4 days
H2O cluster name:,H2O_from_python_unknownUser_gpt3g2
H2O cluster total nodes:,1
H2O cluster free memory:,2.938 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  app.launch_new_instance()
/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
seconds it took 2.154032
Top-10 Hit Ratios: 


k,hit_ratio
1,0.7895002
2,0.8399798
3,0.861686
4,0.8768299
5,0.8828874
6,0.8864210
7,0.8944977
8,0.8985361
9,0.9010600
10,0.9030792


<pre>I am taking only the most important features:
I have selected the features whose percentage of contribution is more than 1%</pre>

In [9]:
importance = model.varimp(use_pandas=True)

importance = importance[importance['percentage']>0.01]

print importance[['variable', 'percentage']]

        variable  percentage
0        country    0.129785
1        natlty1    0.123128
2           iday    0.106729
3    specificity    0.099089
4        success    0.097614
5       extended    0.097484
6     individual    0.096568
7   targsubtype1    0.060893
8       latitude    0.030478
9          iyear    0.012483
10  weapsubtype1    0.011800


<pre>This is what the feature names stand for :

country       The country where the incident occured
natlty1       The nationality of the target that was attacked
iday          Day of the month
specificity   The precision with which the latitude and the longitude are known
success       Whether there was a tangible effect of the attack 
extended      Whether the duration of the attack was more than 24 hours
targsubtype1  The specific category to which the target belongs
latitude      Latitude of the attack
iyear         Year of the attack
weapsubtype1  Specific category of the weapon used 

The selected features "make sense". For example, it is expected that the country the attack
takes place in would be of high importance in the explanation.

The next step is fine-tuning the model with the important features only. First, the set-up:</pre>

In [10]:
#Set-up the model with the important features

training_columns=list(importance['variable'])
response_column = ['gname']

df2 = df[training_columns+ response_column]

random.seed(123)
mysample = random.sample(range(df2.shape[0]), 10000)

df2 = df2.loc[mysample,:]

hf2 = h2o.H2OFrame(df2)

# For the sake of distinguishing between categorical and numeric variables
# I distinguish between numerical variables selected as important:
numericList_imp = list(np.intersect1d(numericList, importance['variable']))


# and the  categoric variables selected as important
categList_imp = list(np.intersect1d(categList, importance['variable']))
#The name of the terror group is also categorical
categList_imp.append('gname')

# Make the distinction between numeric and categoric variables
hf2[:,numericList_imp]=hf2[:,numericList_imp].asnumeric() 
hf2[:,categList_imp] = hf2[:,categList_imp].asfactor()


# Again split into train and test

train, test = hf2.split_frame(ratios=[0.8], seed = 123)


# A function to iterate over the different parameters of the Random Forest model

def EstimateModel( ntrees,  max_depth ):
    t=time.clock()
    global model2
    
    # Initialize the model
    model2 = H2ORandomForestEstimator( ntrees = ntrees, max_depth = max_depth, seed= 123)
    # Train the model
    model2.train(x=training_columns, y=str(response_column[0]), training_frame=train)
    # Predict performance
    performance = model2.model_performance(test_data=test)    
    
    #Display parameters, performance, and the amount of time the analysis took
    print 'ntrees = ', ntrees, 'max_depth=', max_depth, 'Hit ratio=', performance.hit_ratio_table()[1][0]
    print 'time it took', time.clock() - t, 'seconds'
  


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  # This is added back by InteractiveShellApp.init_path()


Parse progress: |█████████████████████████████████████████████████████████| 100%


<pre>The next step I took is iterating over a combination of the parameters to find the best.
I have created a vector of 5 different values for the amount of trees to be grown, 
and a vector of 5 values for the depth parameter. 

However, I do not check each of the 25 possible combinations.
Rather, the initial step is to increment both parameters each iteration.

I will use the hit-ratio as the metric to measure success.  It is the number of 
times that a correct prediciton has been made divided by  the total number of predictions. 
It is an easy-to-communicate parameter. </pre>

In [11]:

# Create a vector
ntrees_list = (8,16,20, 24, 28) 
max_depth_list = (3,5,7,9,10)

for i in range(len(ntrees_list)):
    EstimateModel(ntrees_list[i], max_depth_list[i])

drf Model Build progress: |███████████████████████████████████████████████| 100%
ntrees =  8 max_depth= 3 Hit ratio= 0.79202425
time it took 1.120228 seconds
drf Model Build progress: |███████████████████████████████████████████████| 100%
ntrees =  16 max_depth= 5 Hit ratio= 0.81120646
time it took 1.10418 seconds
drf Model Build progress: |███████████████████████████████████████████████| 100%
ntrees =  20 max_depth= 7 Hit ratio= 0.81272084
time it took 0.876742 seconds
drf Model Build progress: |███████████████████████████████████████████████| 100%
ntrees =  24 max_depth= 9 Hit ratio= 0.8192832
time it took 1.000314 seconds
drf Model Build progress: |███████████████████████████████████████████████| 100%
ntrees =  28 max_depth= 10 Hit ratio= 0.819788
time it took 1.07512 seconds


<pre>The changes in the hit ratio become rather small.

When I decrease the number of trees, it has an additional small positive effect:</pre>

In [12]:
EstimateModel(ntrees=24, max_depth=10)

drf Model Build progress: |███████████████████████████████████████████████| 100%
ntrees =  24 max_depth= 10 Hit ratio= 0.8202928
time it took 1.039864 seconds


 #### The hit-ratio on the validation set  is 82% 

## Final prediction:

<pre>Not all of the terror acts in the GTD database have been identified with a certain group.
The task of the challenge is to predict the groups responsible for those un-identified terror
acts. Though of course it's impossible to verify the truth of their predictions using the
current database, a hit-ratio of 82% on the validation set does look good (especially if we 
take into account the large number of terror groups that are involved.) </pre>

<pre>I use the parameters that gave the highest hit-ratio so far (24 trees, 10 depth), and
make the final prediction</pre>

In [13]:
# Prepare the final dataset for H2O library


hf_all = h2o.H2OFrame(df[training_columns+ response_column])

#Preparing the final test-set
prd_group = prd_group.drop('gname', axis=1)
hf_pred = h2o.H2OFrame(prd_group)


# Differentiate between numeric and categoric variables 
hf_pred[:,numericList_imp]=hf_pred[:,numericList_imp].asnumeric() 
hf_pred[:,categList_imp[:-1]] = hf_pred[:,categList_imp[:-1]].asfactor()

hf_all[:,numericList_imp]=hf_all[:,numericList_imp].asnumeric() 
hf_all[:,categList_imp] = hf_all[:,categList_imp].asfactor()

#initialize model
model_all = H2ORandomForestEstimator( ntrees = 24, max_depth = 10, seed= 123)
# Train the model
model_all.train(x=training_columns, y=str(response_column[0]), training_frame=hf_all)

# Make the prediction
final_pred = model_all.predict(hf_pred)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


/usr/local/lib/python2.7/dist-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'country' has levels not trained on: [10, 17, 22, 23, 35, 63, 64, 72, 73, 80, 85, 91, 109, 115, 117, 120, 124, 129, 132, 143, 180, 181, 189, 219, 223, 226, 377, 428, 1001, 1002]
  warnings.warn(w)
/usr/local/lib/python2.7/dist-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'natlty1' has levels not trained on: [10, 22, 23, 24, 31, 64, 70, 72, 73, 80, 85, 109, 117, 124, 125, 127, 129, 132, 134, 143, 176, 180, 189, 225, 377, 1002]
  warnings.warn(w)


<pre>Though using the current dataset only, it is not possible to verify the final results,
a basic feasability-check can be performed.
I show all the groups identified as being responsible for terror acts in Israel (my country),
grouped by the number of identifications made for each group.</pre>

In [14]:
results = hf_pred.cbind(final_pred['predict']).as_data_frame()
results[results['country'] == 97].loc[:,['predict']].stack().value_counts()

Hamas (Islamic Resistance Movement)                        280
Palestinians                                               244
Palestinian Islamic Jihad (PIJ)                             92
Palestinian Extremists                                      51
Hezbollah                                                   33
Al-Aqsa Martyrs Brigade                                     31
Popular Resistance Committees                               13
Israeli extremists                                           9
Popular Front for the Liberation of Palestine (PFLP)         8
Ansar Bayt al-Maqdis (Ansar Jerusalem)                       4
Democratic Front for the Liberation of Palestine (DFLP)      3
Keshet                                                       2
Palestine Liberation Organization (PLO)                      1
Jewish Extremists                                            1
Black Panther Group (Palestinian)                            1
Tripoli Province of the Islamic State                  

<pre>Many of those group names are familiar to Israeli citizens as groups that performed terror acts in 
 the past. One can also notice diversity in the results - though most of the predictions are of 
Palestinian and Islamic groups, not all are (Keshet,Jewish Extremists, Israeli Settlers). That helps to demonstrate to the non-technical reader the "smartness" of the algorithm. 

All in all, the results appear sensible. 
</pre>

In [15]:
#Shutdown H2O
h2o.cluster().shutdown(prompt = False)

H2O session _sid_ad73 closed.
